In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


RNN(Recurrent Neural Network) - 순환 신경망

RNN은 순차적인 데이터를 입력받아 결과값을 도출하는 데 사용하는 딥러링 모델로 자연어 처리에 상당히 많이 사용되고 이전에 입력된 값들을 고려해서 현재 입력값의 출력값을 결정하는 딥러닝 모델이다.

x는 입력값, y는 출력값, 활성화 함수(tanh)를 거친 값은 상태(출력)값을 의미한다. 네모 박스는 셀이라 하며 셀 안에서 현재 셀의 입력값과 과거 셀의 상태값을 사용해 현재 셀의 상태값을 계산한다.  
현재 셀의 상태값은 현재 셀의 출력값과 동일하며 다음 셀의 이전 상태값으로 사용된다.

상태값을 결정하기 위해 다음 그림과 같이 두 가지의 가중치가 존재한다. 현재 셀의 상태값은 tanh(입력값 * $W_{xh}$ + 이전 셀의 상태값 * $W_{fh}$ + 편향값)으로 결정된다. 가중치와 편향값은 최초 무작위로 부여하고 학습 과정을 통해 가중치 및 평향값은 모델의 목적에 맞게 최적화 된다.

<img src="./images/RNN.png" align="left" width="800"/>

RNN 텐서플로우 코드 구현

In [2]:
inputs = np.array([[[1, 2]]]) # 입력 데이터
# print(inputs) # [[[1 2]]]
# print(inputs.shape) # (1, 1, 2)
# print(inputs.shape[0]) # 1
tf.set_random_seed(35)

tf_inputs = tf.constant(inputs, dtype=tf.float32)
sess = tf.Session()
print('입력 데이터: {}'.format(sess.run(tf_inputs)))

# BasicRNNCell() 메소드의 num_units 속성으로 셀의 개수를 지정해서 RNN 셀을 만든다.
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=4)
print('RNN 셀의 개수: {}'.format(rnn_cell.state_size))

# RNN 셀 초기화
initial_state = tf.random_normal(shape=[inputs.shape[0], rnn_cell.state_size], mean=1.0) # [1, 3]
print(sess.run(initial_state))
print('=' * 80)

# dynamic_rnn() 메소드는 입력값과 RNN 셀을 받아서 실행한 결과(출력값, 상태값)를 리턴한다.
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs, initial_state=initial_state)
print('=' * 80)

print('출력값: {}'.format(outputs))
print('상태값: {}'.format(state))
print('=' * 80)

print('가중치 개수와 편향값의 개수')
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)

입력 데이터: [[[1. 2.]]]
RNN 셀의 개수: 4
[[ 0.3164147   0.68187654  1.6424221  -0.03141737]]
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
출력값: Tensor("rnn/transpose_1:0", shape=(1, 1, 4), dtype=float32)
상태값: Tensor("rnn/while/Exit_3:0", shape=(1, 4), dtype=float32)
가중치 개수와 편향값의 개수
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(6, 4) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(4,) dtype=float32_ref>


In [3]:
variable_name = [vari.name for vari in tf.trainable_variables()]
print(variable_name)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_rnn, state_rnn = sess.run([outputs, state])
    # 출력값과 상태값은 같은 값이고 상태값이 다음 RNN 셀로 전달된다.
    print('출력값: {}'.format(output_rnn))
    print('상태값: {}'.format(state_rnn))
    
    values = sess.run(variable_name)
    print('가중치: ', variable_name[0], '\n', values[0], sep='')
    print('편향값: ', variable_name[1], '\n', values[1], sep='')

['rnn/basic_rnn_cell/kernel:0', 'rnn/basic_rnn_cell/bias:0']
출력값: [[[-0.5809659   0.9154984  -0.9457683  -0.83892715]]]
상태값: [[-0.5809659   0.9154984  -0.9457683  -0.83892715]]
가중치: rnn/basic_rnn_cell/kernel:0
[[-0.5418569   0.0407356  -0.04637361 -0.4669427 ]
 [-0.48771197  0.17693835 -0.33036923 -0.31921875]
 [-0.23701847  0.61090493 -0.6731936  -0.68921477]
 [ 0.00095868  0.4474933  -0.29634744  0.50537777]
 [ 0.5757717   0.40057898 -0.3991353  -0.15756083]
 [ 0.5715413  -0.30284998  0.39252698 -0.6395725 ]]
편향값: rnn/basic_rnn_cell/bias:0
[0. 0. 0. 0.]


RNN 셀은 출력값과 상태값이 동일하다.  
RNN 셀이 1개일 경우 가중치는 3개의 행과 1개의 열을 가지고 편향 값은 1개가 필요하다.  
RNN 셀이 2개일 경우 가중치는 4개의 행과 2개의 열을 가지고 편향 값은 2개가 필요하다.  
RNN 셀이 3개일 경우 가중치는 5개의 행과 3개의 열을 가지고 편향 값은 3개가 필요하다.  
...  
RNN 셀이 n개일 경우 가중치는 n+2개의 행과 n개의 열을 가지고 편향 값은 n개가 필요하다.  